Stand: 10/2020 | Autor: Fabian

# AWS Sagemaker
Sagemaker ist ein Service von AWS zum trainieren von Machine Learning Modellen.
Hierzu erfüllt Sagemaker unterschiedliche Aufgaben. Eine Teil des Services ist die Bereitstellung von Jupyter Notebook Instanzen, die ganz ähnlich wie Jupyter Notebooks auf einem Laptop verwendet werden können. Wobei Sagemaker bei der Verwendung von weiteren AWS Services wie S3 einige Vorteile bietet, welche im folgenden geziegt werden.

Im nächsten Teil wird geziegt, wie die Sagemaker API verwendet werden kann und wie Frameworks wie sklearn und XGB aus  Sagemaker Containern verwendet werden können.


# Einstieg in Sagemaker Notebooks

Ziel dieses Notebooks ist es die die Vorteile von Sagemaker im Zusammenspiel mit anderen AWS Services aufzuzeigen.

Inhalt:

1. Jupyter Magic
2. Einlesen von Dateien
3. Verarbeiten von Daten
4. Visualisieren von Daten
5. Speicher von Dateien


## Importieren von Paketen

Genau wie auf deinem Notebook, kannst du bereits installierte Pakete importieren. Hier besteht __kein__ Unterschied zwischen der Nutzung von Jupyter auf deinem Laptop oder in Sagemaker.


In [ ]:
import pandas as pd
from datetime import datetime

import pickle
import boto3 # Libary für den Zugriff aus AWS Services wie S3
from io import BytesIO

In [ ]:
pd.__version__

## Sagemaker unterstützt Jupyter Magic
Mit den Magic Commands können Behelfe in der Commandline ausgeführt werden.

In [ ]:
! echo hallo

In [ ]:
! pwd

In [ ]:
! python --version

In [ ]:
! pip list

## Einlesen von Daten
Um ein ML Modell trainieren zu können brauchen wir Daten. Welche unterschiedlichen Wege es gibt, Daten zu verwendet wird in diesem Kapitel aufgezeigt.

### Beispieldatensätze aus dem Internet laden

Daten, welche aus dem Internet geladen werden, können wie auf deinem Laptop auch eingelesen und verarbeitet werden. Hier besteht kein Unterschied zu Sagemaker.

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
X, y = load_boston(return_X_y=True)

In [ ]:
print(f'X: {X.shape}, y: {y.shape}')

In [ ]:
y.shape

__Pandas:__

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Fa-bi-an/Intro_Sagemaker/main/data/train.csv')

In [ ]:
df.head()

### S3 Buckets

__1. NICHT serialisierte Dateien direkt aus einem Bucket__<br>
Bspw.: `CSV` oder `JSON`-Dateien. <br>
Hierzu muss das Sagemaker Notebook, über die erforderliche ExecutionRole, also Berechtigungen für den Bucket verfügen!
Diese weist man einem Notebook beim anlegen zu.

In [ ]:
# S3 Muster

bucket ='<bucket-name>'
data_key = '<path>'
file_name = '<file.type>'
data_path = f's3://{bucket}/{data_key}/{file_name}'

In [ ]:
print(f'{data_path}')

In [ ]:
df = pd.read_csv(data_path, sep=';', decimal=',')

In [ ]:
df.head()

__1.2 per Magic Command__

In [ ]:
data_dir = '../data/'
data_path= f's3://{bucket}/{data_key}/{file_name}'

In [ ]:
# Kopiert Dateien aus S3 in den lokalen Ordner der Sagemaker Instanz
!aws s3 cp --recursive $s3_path $data_dir

__2. SERIALISIERTE Dateien__<br>
Bspw.: `pickle`-Dateien.


In [ ]:
import pickle
import boto3
from io import BytesIO
import pandas as pd

In [ ]:
# S3 Muster
bucket ='<bucket-name>'
data_key = '<path>'
file_name = '<file.type>'

In [ ]:
s3 = boto3.resource('s3')
with BytesIO() as data:
    s3.Bucket(bucket).download_fileobj(f"{data_key}/{file_name}", data)
    data.seek(0)    # move back to the beginning after writing
    df = pickle.load(data)

## Verarbeiten von Daten

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
reg = LinearRegression()

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
preds = reg.predict(X_test)

In [ ]:
preds_df = pd.DataFrame(list(zip(preds, y_test)), 
               columns =['Preds', 'y_test']) 

In [ ]:
preds_df.head()

In [ ]:
reg.score(X_test,y_test)

## Visualisiern von Daten

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.scatterplot(x="Preds", y="y_test", data=preds_df )
plt.title('Vorhersage')

## Speichern von Dateien

__1.Speicher von nicht serialisierte Dateien mit Pandas direkt in S3__

In [ ]:
# https://stackoverflow.com/questions/38154040/save-dataframe-to-csv-directly-to-s3-python

from io import StringIO # python3; python2: BytesIO 
import boto3

In [ ]:
bucket = '<bucket-name>' # already created on S3
csv_buffer = StringIO()
preds_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, '<file.type>').put(Body=csv_buffer.getvalue())

__2. lokal Speichern und danach zu S3 hochladen__

In [ ]:
import sagemaker

In [ ]:
session = sagemaker.Session()

In [ ]:
preds_df.to_csv('<file.type>')

In [ ]:
# Datei lokal vorhanden
! ls

In [ ]:
# Verwende die Methode upload_data zum hochladen von lokalen Dateien in S3
session.upload_data(path='<file.type>',bucket='<bucket-name>')

In [ ]:
# Löschen der lokalen Datei
! rm preds.csv

In [ ]:
! ls

__3. serialisierte Dateien zu S3 hochladen__<br>
bspw.: `Pickle`-Dateien

In [ ]:
import io
import os
import boto3

In [ ]:
model_name='<filename>'
model_object=df


def pickle_safe_and_load(model_name,model_object,local_target,s3_target):
    pickle.dump(model_object, open( f"{local_target}/{model_name}.pkl", "wb" ))
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket,f'{s3_target}/{model_name}.pkl').put(Body=open(f"{local_target}/{model_name}.pkl", 'rb'))
    os.remove(f"{local_target}/{model_name}.pkl")